In [519]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from datetime import datetime
import re
import ast

import my_functions
import importlib
importlib.reload(my_functions)

import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [501]:
input_folder = 'input\\'
print(os.listdir("input"))

['main_task.csv']


# DATA
## Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:
* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvosor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [502]:
data = pd.read_csv(input_folder + 'main_task.csv')

data.columns = data.columns.str.lower()
data.columns = [name.replace(' ', '_') for name in data.columns]


# data['price_range'].fillna('$$ - $$$', inplace=True)

data['cuisine_style'] = data['cuisine_style'].fillna('[\'not_specified\']')
data['cuisine_style'] = data['cuisine_style'].apply(ast.literal_eval)

data['reviews'] = data['reviews'].apply(lambda x: re.sub((r'\bnan\b'), '\'empty_voice\'', x))
data['reviews'] = data['reviews'].apply(ast.literal_eval)
data['reviews_text'] =  data['reviews'].apply(lambda x: x[0])
data['reviews_dates'] = data['reviews'].apply(lambda x: [datetime.strptime(date, '%m/%d/%Y') for date in x[1]])

data['number_of_reviews'].fillna(data.number_of_reviews.mean(),inplace=True)
data['price_range_int'] = data['price_range'].apply(lambda x: 0 if x == '$' else (1 if x == '$$ - $$$' else 2))
# data['reviews_dates'] = data['reviews'].apply(lambda x: [pd.to_datetime(date,format='%m/%d/%Y').date() for date in x[1]])


In [523]:
set_dates = set()
data['dif_days'] = data.reviews_dates.apply(lambda x: (x[0] - x[-1]) if len(x) > 0 else pd.Timedelta('0 days') )
data['dif_days'] = data['dif_days'].dt.days

# data['price_range_int'] = data.price_range.apply(lambda x: 0 if x == '')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      40000 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  40000 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  object 
 10  reviews_text       40000 non-null  object 
 11  reviews_dates      40000 non-null  object 
 12  price_range_int    40000 non-null  int64  
 13  dif_days           40000 non-null  int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 4.3+ MB


In [529]:


data.price_range.value_counts(dropna=False)

data.groupby(by='city').agg({'price_range_int':'mean'}, 'city':'size')



SyntaxError: invalid syntax (<ipython-input-529-b85b2aeb6dca>, line 3)

In [505]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)

X = data.drop(['restaurant_id','city', 'rating', 'cuisine_style', 'price_range', 'reviews', 'url_ta', 'id_ta', 'reviews_text','reviews_dates'], axis = 1)
y = data['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train

# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [506]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.39302600286796535
